<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin-bottom: 20px;">
    <h1 style="color: white; margin: 0; font-size: 36px;">⚙️ Notebook 3: Modules & Adapters</h1>
    <p style="color: rgba(255,255,255,0.9); margin-top: 10px; font-size: 18px;">Execution Strategies & Output Formats for LLMs</p>
</div>

<div style="display: flex; justify-content: space-between; margin-bottom: 20px;">
    <a href="02_signatures.ipynb" style="text-decoration: none; padding: 10px 20px; background: #f0f0f0; border-radius: 5px;">← Notebook 2</a>
    <span style="padding: 10px 20px; background: #fff8e1; border-radius: 5px;">🟡 Intermediate • 25 minutes</span>
    <a href="04_providers.ipynb" style="text-decoration: none; padding: 10px 20px; background: #f0f0f0; border-radius: 5px;">Notebook 4 →</a>
</div>

## 🎯 What You'll Learn

<div style="background: #f5f5f5; padding: 20px; border-radius: 10px; border-left: 4px solid #667eea;">
    <h3>Modules - Execution Strategies</h3>
    <ul style="margin: 10px 0; padding-left: 20px;">
        <li>✅ <strong>Predict</strong>: Fast, simple transformations</li>
        <li>✅ <strong>ChainOfThought</strong>: Add reasoning for complex tasks</li>
        <li>✅ <strong>Retry</strong>: Handle failures gracefully</li>
        <li>✅ <strong>Refine</strong>: Iterative improvement</li>
        <li>✅ <strong>Module Composition</strong>: Combine strategies</li>
    </ul>    
    <h3>Adapters - Output Formats</h3>
    <ul style="margin: 10px 0; padding-left: 20px;">
        <li>✅ <strong>JSON</strong>: Structured, reliable parsing</li>
        <li>✅ <strong>XML</strong>: Alternative structured format</li>
        <li>✅ <strong>Markdown</strong>: Human-readable (with caveats)</li>
        <li>✅ <strong>Chat</strong>: Conversational format</li>
        <li>✅ <strong>Choosing the right adapter</strong> for your data</li>
    </ul>
</div>

## 🔧 Setup

In [1]:
import asyncio
import time
import os
from typing import List, Dict, Optional

import logillm
from logillm.core.predict import Predict, ChainOfThought
from logillm.core.retry import Retry
from logillm.core.refine import Refine
from logillm.core.signatures import Signature, InputField, OutputField
from logillm.providers import create_provider, register_provider

# Check API key
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️ WARNING: OPENAI_API_KEY not set!")
    print("Set it with: export OPENAI_API_KEY=your_key")
else:
    print("✅ OpenAI API key found")

# Setup provider
try:
    provider = create_provider("openai", model="gpt-4.1-mini")
    register_provider(provider, set_default=True)
    print(f"✅ LogiLLM {logillm.__version__} ready!")
except Exception as e:
    print(f"❌ Error: {e}")

✅ OpenAI API key found
✅ LogiLLM 0.2.17 ready!


## 📚 Part 1: Understanding Modules

**Modules** are execution strategies that determine HOW your LLM processes signatures.

In [2]:
# Define a signature for testing
class TextAnalysis(Signature):
    """Analyze text for key information."""
    text: str = InputField(desc="Text to analyze")
    
    summary: str = OutputField(desc="Brief summary")
    topics: list[str] = OutputField(desc="Main topics (3-5)")
    sentiment: str = OutputField(desc="positive, negative, or neutral")
    confidence: float = OutputField(desc="Confidence score 0-1")

test_text = """
The new LogiLLM framework is revolutionizing how developers work with large language models.
It provides clean abstractions, reliable parsing, and excellent performance. However, some users
find the initial learning curve challenging. Overall, it's becoming an essential tool.
"""

print("🎯 We'll use this signature to explore different modules and adapters")

🎯 We'll use this signature to explore different modules and adapters


### 🔹 Module 1: Predict - The Foundation

In [3]:
# Basic Predict module
basic_analyzer = Predict(TextAnalysis)

print("🔹 Using Predict (simplest module):")
result = await basic_analyzer(text=test_text)

print(f"\n📝 Summary: {result.outputs['summary']}")
print(f"🏷️ Topics: {', '.join(result.outputs['topics'])}")
print(f"😊 Sentiment: {result.outputs['sentiment']}")
print(f"📊 Confidence: {result.outputs['confidence']}")
print(f"\n✅ All fields guaranteed to exist!")

🔹 Using Predict (simplest module):

📝 Summary: The LogiLLM framework is changing how developers use large language models by offering clean abstractions, reliable parsing, and strong performance, despite a challenging initial learning curve. It is becoming an essential tool.
🏷️ Topics: LogiLLM framework, large language models, software development, performance, learning curve
😊 Sentiment: positive
📊 Confidence: 0.9

✅ All fields guaranteed to exist!


### 🧠 Module 2: ChainOfThought - Adding Reasoning

In [4]:
# ChainOfThought adds reasoning
reasoning_analyzer = ChainOfThought(TextAnalysis)

print("🧠 Using ChainOfThought (adds reasoning):")
result = await reasoning_analyzer(text=test_text)

print(f"\n💭 Reasoning: {result.outputs['reasoning'][:200]}...")
print(f"\n📝 Summary: {result.outputs['summary']}")
print(f"🏷️ Topics: {', '.join(result.outputs['topics'])}")
print(f"😊 Sentiment: {result.outputs['sentiment']}")
print(f"\n✨ Notice: 'reasoning' field was automatically added!")

🧠 Using ChainOfThought (adds reasoning):

💭 Reasoning: Let's think step by step to solve this problem. The text discusses the LogiLLM framework, highlighting its positive aspects such as clean abstractions, reliable parsing, and excellent performance, whi...

📝 Summary: The LogiLLM framework improves work with large language models by offering clean abstractions, reliable parsing, and strong performance, though some users find it initially challenging to learn. Overall, it is becoming an essential tool for developers.
🏷️ Topics: LogiLLM framework, Large language models, Software features, User experience, Developer tools
😊 Sentiment: positive

✨ Notice: 'reasoning' field was automatically added!


## 🎨 Part 2: Understanding Adapters

**Adapters** determine the OUTPUT FORMAT - how the LLM structures its response.

<div style="background: #fff3e0; padding: 15px; border-radius: 8px; margin: 20px 0;">
    <strong>⚠️ Important:</strong> Adapter choice significantly affects parsing reliability, especially for structured outputs with lists!
</div>

### 📊 Adapter Comparison: JSON vs XML vs Markdown

In [5]:
# Test different adapters with the SAME signature
print("🔬 Testing Different Adapters:\n")
print("=" * 60)

# 1. JSON Adapter (RECOMMENDED for structured data)
print("\n1️⃣ JSON ADAPTER:")
json_analyzer = Predict(TextAnalysis, adapter="json")
result_json = await json_analyzer(text=test_text)

print(f"  ✅ Topics parsed: {len(result_json.outputs['topics'])} items")
print(f"  ✅ First topic: {result_json.outputs['topics'][0] if result_json.outputs['topics'] else 'None'}")
print(f"  ✅ Confidence type: {type(result_json.outputs['confidence']).__name__}")
print(f"  ✅ All fields present: {set(result_json.outputs.keys())}")

# 2. XML Adapter (Good alternative)
print("\n2️⃣ XML ADAPTER:")
xml_analyzer = Predict(TextAnalysis, adapter="xml")
result_xml = await xml_analyzer(text=test_text)

print(f"  ✅ Topics parsed: {len(result_xml.outputs['topics'])} items")
print(f"  ✅ First topic: {result_xml.outputs['topics'][0] if result_xml.outputs['topics'] else 'None'}")
print(f"  ✅ Confidence type: {type(result_xml.outputs['confidence']).__name__}")
print(f"  ✅ All fields present: {set(result_xml.outputs.keys())}")

# 3. Markdown Adapter (AVOID for structured data)
print("\n3️⃣ MARKDOWN ADAPTER:")
markdown_analyzer = Predict(TextAnalysis, adapter="markdown")
result_md = await markdown_analyzer(text=test_text)

# Markdown often has parsing issues with lists
topics_md = result_md.outputs.get('topics', [])
if isinstance(topics_md, list) and len(topics_md) > 0:
    print(f"  ⚠️ Topics parsed: {len(topics_md)} items")
    if len(topics_md) > 10:  # Likely parsed as individual characters
        print(f"  ❌ WARNING: Likely parsing error (got {len(topics_md)} items)!")
else:
    print(f"  ❌ Topics: Failed to parse as list")
print(f"  ⚠️ Confidence type: {type(result_md.outputs.get('confidence', 'missing')).__name__}")
print(f"  ⚠️ Fields present: {set(result_md.outputs.keys())}")

print("\n" + "=" * 60)
print("📊 VERDICT: JSON adapter provides most reliable parsing!")

🔬 Testing Different Adapters:


1️⃣ JSON ADAPTER:
  ✅ Topics parsed: 0 items
  ✅ First topic: None
  ✅ Confidence type: NoneType
  ✅ All fields present: {'confidence', 'topics', 'summary', 'sentiment'}

2️⃣ XML ADAPTER:
  ✅ Topics parsed: 5 items
  ✅ First topic: LogiLLM framework
  ✅ Confidence type: str
  ✅ All fields present: {'confidence', 'topics', 'summary', 'sentiment'}

3️⃣ MARKDOWN ADAPTER:
  ❌ Topics: Failed to parse as list
  ⚠️ Confidence type: str
  ⚠️ Fields present: {'confidence', 'topics', 'summary', 'sentiment'}

📊 VERDICT: JSON adapter provides most reliable parsing!


### 🎯 Smart Adapter Selection

In [6]:
# LogiLLM can automatically select the best adapter
print("🤖 Automatic Adapter Selection:\n")

# Signature with lists → JSON automatically selected
auto_analyzer = Predict(TextAnalysis)  # No adapter specified
print(f"Complex signature → Adapter: {auto_analyzer.adapter.format_type.value}")

# Simple signature → Chat adapter selected
class SimpleSignature(Signature):
    text: str = InputField()
    summary: str = OutputField()

simple_analyzer = Predict(SimpleSignature)
print(f"Simple signature → Adapter: {simple_analyzer.adapter.format_type.value}")

print("\n✨ LogiLLM intelligently selects adapters based on signature complexity!")

🤖 Automatic Adapter Selection:

Complex signature → Adapter: json
Simple signature → Adapter: chat

✨ LogiLLM intelligently selects adapters based on signature complexity!


## 🔗 Part 3: Combining Modules & Adapters

The real power comes from combining the right module with the right adapter.

In [7]:
# Complex task requiring both reasoning AND reliable parsing
class CodeReview(Signature):
    """Review code for issues and improvements."""
    code: str = InputField(desc="Code to review")
    language: str = InputField(desc="Programming language")
    
    issues: list[str] = OutputField(desc="List of issues found")
    improvements: list[str] = OutputField(desc="Suggested improvements")
    security_risks: list[str] = OutputField(desc="Security vulnerabilities")
    quality_score: int = OutputField(desc="Quality score 1-10")

sample_code = '''
def get_user(user_id):
    query = f"SELECT * FROM users WHERE id = {user_id}"
    return db.execute(query)
'''

print("🔍 Code Review: Module + Adapter Combinations\n")

# 1. Basic Predict with Markdown (problematic)
print("❌ Predict + Markdown (not recommended):")
basic_md = Predict(CodeReview, adapter="markdown")
result_md = await basic_md(code=sample_code, language="python")
print(f"  Issues found: {len(result_md.outputs.get('issues', []))}")
print(f"  Issue parsing: {'✅ OK' if isinstance(result_md.outputs.get('issues'), list) else '❌ Failed'}")

# 2. ChainOfThought with JSON (recommended)
print("\n✅ ChainOfThought + JSON (recommended):")
smart_json = ChainOfThought(CodeReview, adapter="json")
result_json = await smart_json(code=sample_code, language="python")
print(f"  Issues found: {len(result_json.outputs['issues'])}")
print(f"  First issue: {result_json.outputs['issues'][0] if result_json.outputs['issues'] else 'None'}")
print(f"  Has reasoning: {'reasoning' in result_json.outputs}")
print(f"  Quality score type: {type(result_json.outputs['quality_score']).__name__}")

# 3. Robust pipeline: ChainOfThought + JSON + Retry
print("\n🏆 Production Pipeline (ChainOfThought + JSON + Retry):")
robust_reviewer = Retry(
    ChainOfThought(CodeReview, adapter="json"),
    max_retries=2
)
result_robust = await robust_reviewer(code=sample_code, language="python")
print(f"  All fields parsed: ✅")
print(f"  Issues: {len(result_robust.outputs['issues'])} found")
print(f"  Security risks: {len(result_robust.outputs['security_risks'])} found")
print(f"  With reasoning: ✅")
print(f"  With retry protection: ✅")

🔍 Code Review: Module + Adapter Combinations

❌ Predict + Markdown (not recommended):
  Issues found: 890
  Issue parsing: ❌ Failed

✅ ChainOfThought + JSON (recommended):
  Issues found: 4
  First issue: SQL query is constructed using string formatting which may lead to SQL injection vulnerabilities.
  Has reasoning: True
  Quality score type: int

🏆 Production Pipeline (ChainOfThought + JSON + Retry):
  All fields parsed: ✅
  Issues: 4 found
  Security risks: 1 found
  With reasoning: ✅
  With retry protection: ✅


## 🛠️ Part 4: Advanced Module Patterns

### 🔄 Retry Module - Handling Failures

In [8]:
# Retry module for reliability
from logillm.core.retry import RetryStrategy

class ComplexTask(Signature):
    """A task that might fail or need retries."""
    input: str = InputField()
    constraints: list[str] = InputField(desc="Constraints to satisfy")
    
    solution: str = OutputField(desc="Solution meeting all constraints")
    steps: list[str] = OutputField(desc="Steps taken")
    valid: bool = OutputField(desc="Whether solution meets all constraints")

# Configure retry with different strategies
retry_configs = [
    ("Immediate", RetryStrategy.IMMEDIATE, 0),
    ("Linear", RetryStrategy.LINEAR, 1.0),
    ("Exponential", RetryStrategy.EXPONENTIAL, 1.0)
]

print("🔄 Retry Strategies Comparison:\n")

for name, strategy, base_delay in retry_configs:
    retrier = Retry(
        Predict(ComplexTask, adapter="json"),
        max_retries=2,
        strategy=strategy,
        base_delay=base_delay
    )
    
    print(f"📍 {name} Retry:")
    start = time.time()
    result = await retrier(
        input="Design a system",
        constraints=["scalable", "secure", "cost-effective"]
    )
    elapsed = time.time() - start
    
    print(f"  Time: {elapsed:.2f}s")
    print(f"  Success: {result.success}")
    print(f"  Steps: {len(result.outputs.get('steps', []))} defined")
    print()

🔄 Retry Strategies Comparison:

📍 Immediate Retry:
  Time: 3.53s
  Success: True
  Steps: 6 defined

📍 Linear Retry:
  Time: 4.84s
  Success: True
  Steps: 7 defined

📍 Exponential Retry:
  Time: 3.14s
  Success: True
  Steps: 6 defined



### ✨ Refine Module - Iterative Improvement

In [9]:
# Refine for quality improvement
class CreativeWriting(Signature):
    """Generate creative content."""
    prompt: str = InputField()
    style: str = InputField(desc="Writing style")
    
    content: str = OutputField(desc="Generated content")
    title: str = OutputField(desc="Title")

# Quality evaluation function
def content_quality(inputs: dict, prediction) -> float:
    """Evaluate content quality."""
    if not prediction.success:
        return 0.0
    
    content = prediction.outputs.get('content', '')
    title = prediction.outputs.get('title', '')
    
    score = 0.0
    if title and len(title) > 3:
        score += 0.3
    if len(content) > 100:
        score += 0.4
    if content.count('.') > 2:  # Multiple sentences
        score += 0.3
    
    return min(score, 1.0)

# Create refiner with JSON adapter for reliability
refiner = Refine(
    module=Predict(CreativeWriting, adapter="json"),
    N=3,  # Try 3 times
    reward_fn=content_quality,
    threshold=0.8
)

print("✨ Content Refinement Example:\n")
result = await refiner(
    prompt="AI helping humanity",
    style="inspirational"
)

print(f"📝 Title: {result.outputs['title']}")
print(f"\n{result.outputs['content'][:200]}...")

if 'refinement_attempts' in result.metadata:
    print(f"\n📊 Refined through {result.metadata['refinement_attempts']} attempts")

✨ Content Refinement Example:

📝 Title: A Brighter Tomorrow: AI and Humanity United

In the dawn of a new era, artificial intelligence stands not as a distant marvel, but as a beacon of hope illuminating the path forward. With every algorithm and every line of code, AI extends a helpi...

📊 Refined through 1 attempts


## 📊 Part 5: Performance & Best Practices

In [10]:
# Performance comparison
print("⚡ Module + Adapter Performance Matrix:\n")
print(f"{'Configuration':<30} {'Time (s)':<10} {'Reliability':<12} {'Use Case'}")
print("=" * 75)

configs = [
    ("Predict + Chat", Predict(SimpleSignature), "⭐⭐⭐⭐⭐", "Simple tasks"),
    ("Predict + JSON", Predict(TextAnalysis, adapter="json"), "⭐⭐⭐⭐⭐", "Structured data"),
    ("Predict + Markdown", Predict(TextAnalysis, adapter="markdown"), "⭐⭐", "Human-readable"),
    ("ChainOfThought + JSON", ChainOfThought(TextAnalysis, adapter="json"), "⭐⭐⭐⭐⭐", "Complex reasoning"),
    ("Retry(Predict) + JSON", Retry(Predict(TextAnalysis, adapter="json")), "⭐⭐⭐⭐", "Network issues"),
]

for name, module, reliability, use_case in configs:
    start = time.time()
    try:
        result = await module(text="Test text for analysis.")
        elapsed = time.time() - start
        print(f"{name:<30} {elapsed:<10.2f} {reliability:<12} {use_case}")
    except:
        print(f"{name:<30} {'N/A':<10} {reliability:<12} {use_case}")

⚡ Module + Adapter Performance Matrix:

Configuration                  Time (s)   Reliability  Use Case
Predict + Chat                 21.52      ⭐⭐⭐⭐⭐        Simple tasks
Predict + JSON                 1.25       ⭐⭐⭐⭐⭐        Structured data
Predict + Markdown             1.15       ⭐⭐           Human-readable
ChainOfThought + JSON          1.81       ⭐⭐⭐⭐⭐        Complex reasoning
Retry(Predict) + JSON          0.93       ⭐⭐⭐⭐         Network issues


## 🏗️ Part 6: Production Pipeline Example

In [11]:
# Production-ready document processing pipeline
class DocumentProcessor(Signature):
    """Process and analyze documents."""
    document: str = InputField(desc="Document text")
    doc_type: str = InputField(desc="Type: email, report, article")
    requirements: list[str] = InputField(desc="Processing requirements")
    
    summary: str = OutputField(desc="Executive summary")
    key_points: list[str] = OutputField(desc="Main points")
    action_items: list[str] = OutputField(desc="Action items if any")
    entities: list[str] = OutputField(desc="Named entities mentioned")
    priority: str = OutputField(desc="high, medium, or low")
    tags: list[str] = OutputField(desc="Relevant tags")

def create_production_pipeline():
    """Create a robust, production-ready pipeline."""
    
    # Step 1: Choose the right module (ChainOfThought for complex analysis)
    # Step 2: Choose the right adapter (JSON for structured outputs)
    # Step 3: Add reliability (Retry wrapper)
    
    pipeline = Retry(
        ChainOfThought(
            DocumentProcessor,
            adapter="json"  # Critical: JSON for list fields!
        ),
        max_retries=3,
        strategy=RetryStrategy.EXPONENTIAL,
        base_delay=1.0
    )
    
    return pipeline

# Test the pipeline
pipeline = create_production_pipeline()

test_doc = """
Subject: Q4 Strategy Meeting - Urgent

Team,

Following our discussion with John Smith from Marketing and Sarah Johnson from Engineering,
we need to pivot our Q4 strategy. The new AI features are showing great promise but need
more testing. 

Key decisions needed:
- Delay launch by 2 weeks for additional QA
- Increase marketing budget by 20%
- Hire 3 additional engineers

Please review and respond by Friday.

Best,
Mike Chen
Product Manager
"""

print("🏭 Production Pipeline Test:\n")
result = await pipeline(
    document=test_doc,
    doc_type="email",
    requirements=["extract action items", "identify people", "assess urgency"]
)

# All fields guaranteed to exist and be properly parsed!
print(f"📝 Summary: {result.outputs['summary']}\n")

print("👥 Entities:")
for entity in result.outputs['entities'][:5]:
    print(f"  • {entity}")

print("\n✅ Action Items:")
for item in result.outputs['action_items']:
    print(f"  • {item}")

print(f"\n🚨 Priority: {result.outputs['priority'].upper()}")
print(f"🏷️ Tags: {', '.join(result.outputs['tags'])}")

print("\n✨ Benefits of this pipeline:")
print("  • JSON adapter ensures lists parse correctly")
print("  • ChainOfThought provides reasoning for decisions")
print("  • Retry handles transient failures")
print("  • All output fields guaranteed to exist")

🏭 Production Pipeline Test:

📝 Summary: The email from Mike Chen, Product Manager, addresses the need to pivot the Q4 strategy following discussions with John Smith from Marketing and Sarah Johnson from Engineering. Key strategic changes include delaying the product launch for extra QA testing, increasing the marketing budget by 20%, and hiring three additional engineers. The team is requested to review and respond by Friday, highlighting the urgency of these decisions.

👥 Entities:
  • Mike Chen
  • John Smith
  • Sarah Johnson
  • Marketing
  • Engineering

✅ Action Items:
  • Delay launch by 2 weeks for additional QA.
  • Increase marketing budget by 20%.
  • Hire 3 additional engineers.
  • Team to review changes and respond by Friday.

🚨 Priority: HIGH
🏷️ Tags: Q4 strategy, AI features, product launch, marketing budget, hiring, QA, urgent

✨ Benefits of this pipeline:
  • JSON adapter ensures lists parse correctly
  • ChainOfThought provides reasoning for decisions
  • Retry handl

## 🎯 Key Takeaways

<div style="background: #e8f5e9; padding: 25px; border-radius: 10px; margin: 20px 0;">
    <h3 style="margin-top: 0;">Module Selection Guide</h3>
    <table style="width: 100%; margin: 15px 0;">
        <tr style="background: rgba(0,0,0,0.05);">
            <th style="padding: 10px; text-align: left;">Module</th>
            <th style="padding: 10px;">When to Use</th>
        </tr>
        <tr>
            <td style="padding: 8px;"><strong>Predict</strong></td>
            <td style="padding: 8px;">Simple transformations, speed critical</td>
        </tr>
        <tr style="background: rgba(0,0,0,0.02);">
            <td style="padding: 8px;"><strong>ChainOfThought</strong></td>
            <td style="padding: 8px;">Complex reasoning, multi-step problems</td>
        </tr>
        <tr>
            <td style="padding: 8px;"><strong>Retry</strong></td>
            <td style="padding: 8px;">Network reliability, parsing complex data</td>
        </tr>
        <tr style="background: rgba(0,0,0,0.02);">
            <td style="padding: 8px;"><strong>Refine</strong></td>
            <td style="padding: 8px;">Quality over speed, creative tasks</td>
        </tr>
    </table>    
    <h3 style="margin-top: 20px;">Adapter Selection Guide</h3>
    <table style="width: 100%; margin: 15px 0;">
        <tr style="background: rgba(0,0,0,0.05);">
            <th style="padding: 10px; text-align: left;">Adapter</th>
            <th style="padding: 10px;">Best For</th>
            <th style="padding: 10px;">Avoid For</th>
        </tr>
        <tr>
            <td style="padding: 8px;"><strong>JSON</strong> ⭐</td>
            <td style="padding: 8px;">Structured data, lists, production</td>
            <td style="padding: 8px;">Human-only reading</td>
        </tr>
        <tr style="background: rgba(0,0,0,0.02);">
            <td style="padding: 8px;"><strong>XML</strong></td>
            <td style="padding: 8px;">Alternative to JSON, legacy systems</td>
            <td style="padding: 8px;">Simple outputs</td>
        </tr>
        <tr>
            <td style="padding: 8px;"><strong>Markdown</strong> ⚠️</td>
            <td style="padding: 8px;">Human-readable reports</td>
            <td style="padding: 8px;">Lists, structured data</td>
        </tr>
        <tr style="background: rgba(0,0,0,0.02);">
            <td style="padding: 8px;"><strong>Chat</strong></td>
            <td style="padding: 8px;">Conversations, simple outputs</td>
            <td style="padding: 8px;">Complex structures</td>
        </tr>
    </table>    
    <h3 style="margin-top: 20px;">🏆 Best Practices</h3>
    <ol>
        <li><strong>Use JSON adapter</strong> for any signature with list fields</li>
        <li><strong>Add ChainOfThought</strong> when reasoning improves accuracy</li>
        <li><strong>Wrap with Retry</strong> for production reliability</li>
        <li><strong>Test adapter output</strong> before deploying</li>
        <li><strong>Let LogiLLM auto-select</strong> adapters when unsure</li>
    </ol>
</div>

## 🎮 Interactive Exercise

In [12]:
# Exercise: Build your own optimized pipeline
# Task: Create a customer feedback analyzer

class CustomerFeedback(Signature):
    """Analyze customer feedback comprehensively."""
    feedback: str = InputField(desc="Customer feedback text")
    product: str = InputField(desc="Product name")
    
    sentiment: str = OutputField(desc="positive, negative, neutral")
    issues: list[str] = OutputField(desc="Problems mentioned")
    suggestions: list[str] = OutputField(desc="Improvements suggested")
    satisfaction_score: int = OutputField(desc="1-10 satisfaction")
    follow_up_needed: bool = OutputField(desc="Requires follow-up")
    categories: list[str] = OutputField(desc="Feedback categories")

# TODO: Create your optimized pipeline
# Hints:
# 1. Use ChainOfThought for better analysis
# 2. Use JSON adapter for reliable list parsing
# 3. Add Retry for production reliability

# Your code here:
feedback_analyzer = Retry(
    ChainOfThought(
        CustomerFeedback,
        adapter="json"  # Critical for lists!
    ),
    max_retries=2
)

# Test with sample feedback
test_feedback = """
I've been using your LogiLLM framework for 2 months now. The documentation 
is excellent and the API is intuitive. However, I'm having issues with the 
markdown adapter not parsing lists correctly. It would be great if you could 
add more examples for complex signatures. Also, the JSON adapter works perfectly 
but isn't well documented. Overall, I'm satisfied but these issues are frustrating.
"""

result = await feedback_analyzer(
    feedback=test_feedback,
    product="LogiLLM"
)

print("🎯 Customer Feedback Analysis:\n")
print(f"😊 Sentiment: {result.outputs['sentiment']}")
print(f"📊 Satisfaction: {result.outputs['satisfaction_score']}/10")
print(f"🚨 Follow-up needed: {result.outputs['follow_up_needed']}")

print("\n🐛 Issues:")
for issue in result.outputs['issues']:
    print(f"  • {issue}")

print("\n💡 Suggestions:")
for suggestion in result.outputs['suggestions']:
    print(f"  • {suggestion}")

print("\n🏷️ Categories:", ', '.join(result.outputs['categories']))

if 'reasoning' in result.outputs:
    print(f"\n💭 Analysis reasoning: {result.outputs['reasoning'][:200]}...")

🎯 Customer Feedback Analysis:

😊 Sentiment: positive
📊 Satisfaction: 7/10
🚨 Follow-up needed: True

🐛 Issues:
  • Markdown adapter not parsing lists correctly
  • Lack of examples for complex signatures
  • JSON adapter not well documented

💡 Suggestions:
  • Add more examples for complex signatures
  • Improve documentation for JSON adapter
  • Fix markdown adapter list parsing

🏷️ Categories: documentation, functionality, user experience

💭 Analysis reasoning: The customer feedback mentions both positive and negative aspects of the LogiLLM product. They appreciate the good documentation and intuitive API, highlighting satisfaction with the JSON adapter's fu...


## 🏁 Progress Check

In [13]:
# Progress tracker
completed = {
    "modules_basics": True,
    "adapters_intro": True,
    "adapter_comparison": True,
    "module_adapter_combo": True,
    "retry_patterns": True,
    "refine_patterns": True,
    "performance": True,
    "production_pipeline": True,
    "exercise": True
}

total = len(completed)
done = sum(completed.values())
percentage = (done / total) * 100

print(f"📊 Notebook Progress: {done}/{total} sections ({percentage:.0f}%)")
print("\n" + "█" * int(percentage // 5) + "░" * (20 - int(percentage // 5)))

if percentage == 100:
    print("\n🎉 Excellent! You've mastered Modules & Adapters!")
    print("\n🔑 Key insights gained:")
    print("  • JSON adapter is best for structured data")
    print("  • ChainOfThought adds valuable reasoning")
    print("  • Module composition creates robust pipelines")
    print("  • Adapter choice significantly affects reliability")
    print("\nReady for Notebook 4: Providers!")

📊 Notebook Progress: 9/9 sections (100%)

████████████████████

🎉 Excellent! You've mastered Modules & Adapters!

🔑 Key insights gained:
  • JSON adapter is best for structured data
  • ChainOfThought adds valuable reasoning
  • Module composition creates robust pipelines
  • Adapter choice significantly affects reliability

Ready for Notebook 4: Providers!


<div style="display: flex; justify-content: space-between; margin-top: 40px; padding: 20px; background: #f5f5f5; border-radius: 10px;">
    <a href="02_signatures.ipynb" style="text-decoration: none; padding: 10px 20px; background: white; border-radius: 5px; border: 1px solid #ddd;">← Notebook 2</a>
    <div style="text-align: center;">
        <strong>Congratulations! You understand Modules & Adapters! 🎓</strong>
    </div>
    <a href="04_providers.ipynb" style="text-decoration: none; padding: 10px 20px; background: #667eea; color: white; border-radius: 5px;">Continue to Notebook 4 →</a>
</div>